# Hypothesis and Statistical Testing
---

Import libraries:

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import pingouin as pg

---

## Import cleaned data parquet file

Import the cleaned data that was the output of the 1st notebook

In [2]:
df = pd.read_parquet("../data/mental_health_social_media_dataset_cleaned.parquet")

---
## Hypothesises to test

- Sleep vs Age Group
  - H₀: Mean sleep hours are equal across age groups
  - H₁: At least one group differs
- Stress vs Platform
  - H₀: Mean stress is equal across platforms
  - H₁: At least one platform differs
- Platform × Mental State
  - H₀: No association between platform and mental state
  - H₁: They are associated
- Sleep vs Mental State
  - H₀: Sleep is equal across mental states
  - H₁: At least one mental_state differs
- Screen Time vs Stress
  - Correlation
- Negative Interaction Ratio vs Stress
  - Correlation